## Data Generation

In [1]:
import os
import pandas as pd

In [2]:
input_file = r'../data/yahoo_top_products_click_popularity_20200201.txt'

In [3]:
df_train = pd.read_csv(input_file, sep=r'\t')

# temp descriptions
#df_train.head()

C:\Users\echon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,Parent L1 Description,Parent L2 Description,MID,Merchant Name,OID,Title,Description,Manufacturer,Price,Min Offer Price 30 Day,Price Change,Click Popularity
0,Pet Supplies,More Pet Supplies,248942,Chewy.com,9151715234,Wisdom Panel Health Breed & Health Identificat...,Wisdom Panel Health Breed & Health Identificat...,Wisdom Panel,149.99,89.99,66.67,35415.0480
1,Clothing & Accessories,Men's Clothing,313027,eBay PLA US,11114337071,Girl Scout Cookies 2019-20 New Cookies are in!...,Girl Scout Cookies 2019-20 New Cookies are in!...,Girl Scouts,48.00,48.00,0.00,12215.2644
2,Clothing & Accessories,Men's Clothing,134716,MensUSA.com,5651235052,Classic Long Royal Blue Fashion Zoot Suit,"""This Zoot Suit is as nice and unique as it ge...",mensusa,139.00,139.00,0.00,8317.7750
3,'N/A','N/A',76071,Joe's New Balance Outlet,9756405838,New Balance Women's FuelCore NERGIZE Shoes Bla...,Slip on the FuelCore NERGIZE women's training ...,New Balance,38.99,38.99,0.00,6132.8540
4,Clothing & Accessories,Handbags & Luggage,31851,Kohl's,5681651813,"Stone & Co. Irene Leather Hobo, Grey",Watch the product video here. Stone & Co. embo...,Stone & Co.,99.00,69.30,42.85,5251.5413


In [4]:
# Null Preprocessing
null_rows = (df_train['Title'].isnull()) \
            | (df_train['Parent L1 Description'].isnull()) \
            | (df_train['Parent L2 Description'].isnull()) \
            | (df_train['Price'].isnull()) \
            | (df_train['Click Popularity'].isnull())
df_train = df_train[~null_rows]

# MAX LENGTH substitution
#np.max(df_train['Title'].apply(lambda x : len(x)).to_list())
MAX_LENGTH = 256
df_train = df_train[~(df_train.Title.apply(lambda x : len(x)) > MAX_LENGTH)]

# Sampling
SAMPLE_FRAC = 0.1
df_train = df_train.sample(frac=SAMPLE_FRAC, random_state=9487)

# Column Selections
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['Title', 'Price', 'Click Popularity']]
df_train.head()

,Title,Price,Click Popularity
0,Buy Art For Less Coastal Nautical Sea Life Sea...,48.99,0.1932
1,Safavieh Light Blue/Dark Blue Adirondack 10 ft...,68.50,0.5798
2,Funny Prank Blurry Eye Chart Exam T Shirt,16.99,0.1932
3,"Opinion Ciatti Gaga 35.43"" H x 45.28"" W Desk H...",1369.99,0.7730
4,J. Crew Wool Cardigan Sweater: Black Solid Swe...,21.99,0.3865


In [5]:
# idempotence
df_train.to_csv("../data/train.tsv", sep="\t", index=False)

print("sample number：", len(df_train))
df_train.head()

sample number： 170023


,Title,Price,Click Popularity
0,Buy Art For Less Coastal Nautical Sea Life Sea...,48.99,0.1932
1,Safavieh Light Blue/Dark Blue Adirondack 10 ft...,68.50,0.5798
2,Funny Prank Blurry Eye Chart Exam T Shirt,16.99,0.1932
3,"Opinion Ciatti Gaga 35.43"" H x 45.28"" W Desk H...",1369.99,0.7730
4,J. Crew Wool Cardigan Sweater: Black Solid Swe...,21.99,0.3865


In [ ]:
# Test Dataset Generation
## TODO

## BERT-format + Loader

In [6]:
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset

In [7]:
PRETRAINED_MODEL_NAME = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [8]:
class TargetTextDataset(Dataset):
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]
        self.mode = mode
        self.df = pd.read_csv("../data/" + mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.tokenizer = tokenizer
        
    def __getitem__(self, idx):
        if self.mode == "test":
            text, price = self.df.iloc[idx, :2].values
            reg_score_tensor = None
        else:
            text, price, reg_score = self.df.iloc[idx, :].values
            reg_score_tensor = torch.tensor(reg_score)
            
        # BERT tokens + [SEP]
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        len_text = len(word_pieces)
        
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        segments_tensor = torch.tensor([0] * len_text, dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, reg_score_tensor)
    
    def __len__(self):
        return self.len

In [9]:
trainset = TargetTextDataset("train", tokenizer=tokenizer)

In [10]:
def check_diff_dataset(dataset, index=0):
    # 選擇第一個樣本
    sample_idx = index

    # 將原始文本拿出做比較
    text_a, price_a, reg_score_a = trainset.df.iloc[sample_idx].values

    # 利用剛剛建立的 Dataset 取出轉換後的 id tensors
    tokens_tensor, segments_tensor, reg_score_tensor = trainset[sample_idx]

    # 將 tokens_tensor 還原成文本
    tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
    combined_text = " ".join(tokens)

    # 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
    print(f"""[原始文本]
    句子 1：{text_a}
    句子 2：{price_a}
    分類  ：{reg_score_a}

    --------------------

    [Dataset 回傳的 tensors]
    tokens_tensor  ：{tokens_tensor}

    segments_tensor：{segments_tensor}

    label_tensor   ：{reg_score_tensor}

    --------------------

    [還原 tokens_tensors]
    {combined_text}
    """)

In [11]:
check_diff_dataset(trainset)

[原始文本]
    句子 1：Buy Art For Less Coastal Nautical Sea Life Sea Shell Typography 'Pamper Yourself Dream' by Jean Plout Framed Graphic Art Print IF JP1777 16x16 White 1.5 SM
    句子 2：48.99
    分類  ：0.1932

    --------------------

    [Dataset 回傳的 tensors]
    tokens_tensor  ：tensor([  101,  4965,  2396,  2005,  2625,  5780, 11339,  2712,  2166,  2712,
         5806,  5939,  6873, 12565,  1005, 14089,  4842,  4426,  3959,  1005,
         2011,  3744, 20228,  5833, 10366,  8425,  2396,  6140,  2065, 16545,
        16576,  2581,  2581,  2385,  2595, 16048,  2317,  1015,  1012,  1019,
        15488,   102])

    segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

    label_tensor   ：0.1932000070810318

    --------------------

    [還原 tokens_tensors]
    [CLS] buy art for less coastal nautical sea life sea shell ty ##po ##graphy ' pam ##per yourself dream ' by jean pl ##out framed g

In [12]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [13]:
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        reg_score = torch.stack([s[2] for s in samples])
    else:
        reg_score = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, reg_score


# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 64
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [14]:
def check_dataloader(dataloader):
    data = next(iter(dataloader))

    tokens_tensors, segments_tensors, \
        masks_tensors, reg_score = data

    print(f"""
    tokens_tensors.shape   = {tokens_tensors.shape} 
    {tokens_tensors}
    ------------------------
    segments_tensors.shape = {segments_tensors.shape}
    {segments_tensors}
    ------------------------
    masks_tensors.shape    = {masks_tensors.shape}
    {masks_tensors}
    ------------------------
    label_ids.shape        = {reg_score.shape}
    {reg_score}
    """)

In [15]:
check_dataloader(trainloader)


    tokens_tensors.shape   = torch.Size([64, 46]) 
    tensor([[  101,  4965,  2396,  ...,     0,     0,     0],
        [  101,  7842,  7011,  ...,     0,     0,     0],
        [  101,  6057, 26418,  ...,     0,     0,     0],
        ...,
        [  101, 27327, 11721,  ...,     0,     0,     0],
        [  101,  2033,  5004,  ...,     0,     0,     0],
        [  101, 28576, 23858,  ...,     0,     0,     0]])
    ------------------------
    segments_tensors.shape = torch.Size([64, 46])
    tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
    ------------------------
    masks_tensors.shape    = torch.Size([64, 46])
    tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 

## Modeling
- Huggingface

In [16]:
# 載入一個可以做中文多分類任務的模型，n_class = 3
from transformers import BertForSequenceClassification

#PRETRAINED_MODEL_NAME = "bert-base-chinese"
NUM_LABELS = 1

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

#clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=1, bias=True)


In [17]:
model.config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_labels": 1,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pruned_heads": {},
  "repetition_penalty": 1.0,
  "temperature": 1.0,
  "top_k": 50,
  "top_p": 1.0,
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,

In [ ]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

In [ ]:
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)

        self.init_weights()

    @add_start_docstrings_to_callable(BERT_INPUTS_DOCSTRING)
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.BertConfig`) and inputs:
        loss (:obj:`torch.FloatTensor` of shape :obj:`(1,)`, `optional`, returned when :obj:`label` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
    Examples::
        from transformers import BertTokenizer, BertForSequenceClassification
        import torch
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]
        """

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)